# Titanic Dataset — Preprocessing & Baseline Model

## Objective
Prepare the raw Titanic dataset for machine learning by handling missing values, encoding categorical variables, and splitting into train/test sets. Then train a Logistic Regression baseline model and evaluate its performance.

## Output
At the end of this notebook, we will have:
- A complete preprocessing pipeline (imputation + encoding)
- A trained Logistic Regression baseline model
- Predictions, probabilities, and evaluation metrics (accuracy, confusion matrix, classification report)

## 1. Import Libraries and Load Data

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('data/train.csv')
print(df.shape)
df.head(2)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## 2. Define Label and Features

In [15]:
# Label (target variable)
y = df['Survived']

# Features (all columns except the label)
X = df.drop(columns=['Survived'])

print(f"Label (y): {y.shape}")
print(f"Features (X): {X.shape}")
print(f"\nFeature columns:\n{list(X.columns)}")
X.head()

Label (y): (891,)
Features (X): (891, 11)

Feature columns:
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 3. Drop Non-Useful Columns

Some columns are identifiers (PassengerId, Name, Ticket) or too sparse to use directly (Cabin — 77% missing). Keeping them would add noise without helping the model learn meaningful patterns.

In [16]:
columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']

X = X.drop(columns=columns_to_drop)

print(f"Dropped: {columns_to_drop}")
print(f"Remaining features: {list(X.columns)}")
print(f"Shape: {X.shape}")
X.head()

Dropped: ['PassengerId', 'Name', 'Ticket', 'Cabin']
Remaining features: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
Shape: (891, 7)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


## 4. Train/Test Split

We hold out 20% of the data for testing so we can evaluate the model on data it has never seen. We use `stratify=y` to ensure the survival ratio is preserved in both sets — important because our target is imbalanced.

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set:  X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Test set:      X_test  {X_test.shape},  y_test  {y_test.shape}")
print(f"\nSurvival ratio in full data:     {y.mean():.4f}")
print(f"Survival ratio in training set: {y_train.mean():.4f}")
print(f"Survival ratio in test set:     {y_test.mean():.4f}")

Training set:  X_train (712, 7), y_train (712,)
Test set:      X_test  (179, 7),  y_test  (179,)

Survival ratio in full data:     0.3838
Survival ratio in training set: 0.3834
Survival ratio in test set:     0.3855


## 5. Preprocessing Strategy

### Imputation Plan (Handling Missing Values)
| Column | Strategy | Why |
|---|---|---|
| **Age** | Fill with **median** (from training set) | Median is robust to outliers unlike mean. We use the training set median to avoid data leakage from the test set. |
| **Embarked** | Fill with **mode** (from training set) | Only 2 values missing. Mode (most frequent value) is the safest choice for a categorical column. |

### Encoding Plan (Converting Text to Numbers)
| Column | Strategy | Why |
|---|---|---|
| **Sex** | **Binary encoding** (male=0, female=1) | Only 2 categories, so a single 0/1 column is enough. |
| **Embarked** | **One-hot encoding** (S, C, Q → 3 columns) | 3 categories with no natural order, so one-hot avoids implying a ranking between them. |

## 6. Implement Preprocessing (Pipeline Approach)

### 6A) Identify Column Types

In [18]:
numeric_cols = ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']
categorical_cols = ['Sex', 'Embarked']

print(f"Numeric columns:     {numeric_cols}")
print(f"Categorical columns: {categorical_cols}")
print(f"Total: {len(numeric_cols) + len(categorical_cols)} features")

Numeric columns:     ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']
Categorical columns: ['Sex', 'Embarked']
Total: 7 features


### 6B) Build Preprocessing Transformer

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Numeric pipeline: fill missing values with median
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Categorical pipeline: fill missing values with most frequent, then one-hot encode
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))
])

# Combine both pipelines into one preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

print("Preprocessor created successfully!")
print(preprocessor)

Preprocessor created successfully!
ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['Sex', 'Embarked'])])


## 7. Fit and Transform

In [20]:
# Step 1: Learn from training data + transform it
X_train_processed = preprocessor.fit_transform(X_train)

# Step 2: Transform test data using what was learned from training
X_test_processed = preprocessor.transform(X_test)

print(f"X_train_processed shape: {X_train_processed.shape}")
print(f"X_test_processed shape:  {X_test_processed.shape}")
print(f"\nColumns match: {X_train_processed.shape[1] == X_test_processed.shape[1]}")
print(f"NaNs in train: {np.isnan(X_train_processed).sum()}")
print(f"NaNs in test:  {np.isnan(X_test_processed).sum()}")

X_train_processed shape: (712, 8)
X_test_processed shape:  (179, 8)

Columns match: True
NaNs in train: 0
NaNs in test:  0


## 8. Train a Baseline Model

We start with **Logistic Regression** — a simple, interpretable model that works well for binary classification. It serves as our baseline: any future model should beat this score to be worth the added complexity.

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_processed, y_train)

print("Model trained successfully!")

Model trained successfully!


## 9. Predict on Test Set

In [22]:
y_pred = model.predict(X_test_processed)
y_pred_proba = model.predict_proba(X_test_processed)

print(f"Predictions shape: {y_pred.shape}")
print(f"Probabilities shape: {y_pred_proba.shape}")
print(f"\nFirst 10 predictions:              {y_pred[:10]}")
print(f"First 10 actual:                   {y_test.values[:10]}")
print(f"First 10 survival probabilities:   {np.round(y_pred_proba[:10, 1], 4)}")

# Show side-by-side comparison
results = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred,
    'P(Died)': np.round(y_pred_proba[:, 0], 4),
    'P(Survived)': np.round(y_pred_proba[:, 1], 4)
})
results.head(15)

Predictions shape: (179,)
Probabilities shape: (179, 2)

First 10 predictions:              [0 0 0 0 1 0 1 0 0 0]
First 10 actual:                   [0 0 1 0 1 1 1 0 0 0]
First 10 survival probabilities:   [0.0702 0.0492 0.1581 0.0368 0.6678 0.4352 0.7442 0.3282 0.3502 0.1639]


,Actual,Predicted,P(Died),P(Survived)
0,0,0,0.9298,0.0702
1,0,0,0.9508,0.0492
2,1,0,0.8419,0.1581
3,0,0,0.9632,0.0368
4,1,1,0.3322,0.6678
5,1,0,0.5648,0.4352
6,1,1,0.2558,0.7442
7,0,0,0.6718,0.3282
8,0,0,0.6498,0.3502
9,0,0,0.8361,0.1639


## 10. Model Evaluation

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# --- Accuracy ---
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")
print(f"That means the model correctly predicted {int(accuracy * len(y_test))} out of {len(y_test)} passengers.\n")

# --- Confusion Matrix ---
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print(f"""
  - True Negatives  (correctly predicted died):     {cm[0][0]}
  - False Positives (predicted survived, actually died): {cm[0][1]}
  - False Negatives (predicted died, actually survived): {cm[1][0]}
  - True Positives  (correctly predicted survived): {cm[1][1]}
""")

# --- Classification Report ---
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Died', 'Survived']))

Accuracy: 0.8045 (80.45%)
That means the model correctly predicted 144 out of 179 passengers.

Confusion Matrix:
[[98 12]
 [23 46]]

  - True Negatives  (correctly predicted died):     98
  - False Positives (predicted survived, actually died): 12
  - False Negatives (predicted died, actually survived): 23
  - True Positives  (correctly predicted survived): 46

Classification Report:
              precision    recall  f1-score   support

        Died       0.81      0.89      0.85       110
    Survived       0.79      0.67      0.72        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179



## Model Evaluation Summary

The Logistic Regression baseline model achieved an accuracy of approximately 80% on the test set, indicating that it learned meaningful patterns from the data.

The confusion matrix shows that the model performs very well at identifying passengers who did not survive, correctly classifying most deaths. However, it struggles more with identifying survivors, as reflected by a higher number of false negatives.

From the classification report, the model has high recall for the "Died" class but lower recall for the "Survived" class. This suggests that the model is conservative and tends to predict death unless it is confident about survival.

Overall, this baseline model provides a solid starting point. Future improvements could focus on better capturing survivor-related patterns through feature engineering or alternative models.